<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/examples/experiments/finetune/ADMET_10_30_scale_multiple_run_test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
print(torch.__version__)
print('cuda: ', torch.cuda.is_available())

2.1.0+cu118
cuda:  True


In [ ]:
! pip install rdkit --quiet
! pip install PyTDC --quiet
! pip install mycolorpy --quiet
! pip install selfies  --quiet
! pip install pubchempy --quiet
! pip install dgllife --quiet
! pip install molvs --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet

! pip install DeepPurpose --quiet
! pip install git+https://github.com/bp-kelley/descriptastorus --quiet
! pip install pandas-flavor --quiet

### Code

In [ ]:
from os import walk
import os
file_types = ['bin', 'pth']

# clean certain type of file in path
def clean_files(path='/content/drive/MyDrive/ADMET/', file_types = ['pth']):
    files = next(walk(path), (None, None, []))[2]
    for file in files:
        if isinstance(file, str):
            file_type = file.split('.')[-1]
            if file_type in file_types:
                os.remove(file); print(f'{file} removed from {path}')
# clean_files()

In [ ]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


In [ ]:
# test scripts.func_utils.py

from scripts.func_utils import make_path, convert_with_qed_sa, get_min, \
                                plot_loss, plot_performance

from scripts.eval_utils import *
from scripts.preprocess_mols import *
from scripts.model_architecture import *
from scripts.dataset import *
from scripts.train import *
from tdc.single_pred import ADME
from tdc.single_pred import Tox
from tdc.utils import retrieve_label_name_list
import pandas as pd

label_list = retrieve_label_name_list('herg_central')

def collect_data_10_27(names:list, clean_mol_=False, verbose=False):
    if isinstance(names, str): names = [names]
    name_adme = ['Caco2_Wang', 'Lipophilicity_AstraZeneca',
                 'HydrationFreeEnergy_FreeSolv',
                 'Solubility_AqSolDB'] # regression task
    name_adme+= ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
                'CYP1A2_Veith', 'CYP2C9_Veith'] + \
                ['BBB_Martins', 'Bioavailability_Ma', 'Pgp_Broccatelli',
                 'HIA_Hou','PAMPA_NCATS'] # classify
    print('collect data for: ', names)
    label_list = retrieve_label_name_list('herg_central')
    for i, name in enumerate(names):
        if verbose: print('*'*15, name, '*'*15)
        if name in label_list: data = Tox(name='herg_central', label_name=name)
        elif name in name_adme: data = ADME(name=name)
        else:
            try: data = Tox(name=name)
            except: print('cannot read data!'); return
            if verbose: data.label_distribution()
            # data.label_distribution()
        split = data.get_split()
        train, valid, test = split['train'], split['valid'], split['test']
        if clean_mol_:
            train, valid, test = clean_mol(train), clean_mol(valid), clean_mol(test)

        train = rename_cols(train[['Drug', 'Y']], name)
        valid = rename_cols(valid[['Drug', 'Y']], name)
        test  = rename_cols(test[['Drug', 'Y']],  name)

        if i == 0: trains, valids, tests = train.copy(), valid.copy(), test.copy()
        else:
            trains = trains.merge(train, how='outer')
            valids = valids.merge(valid, how='outer')
            tests = tests.merge(test, how='outer')

    return trains, valids, tests
# trains, valids, tests = collect_data_10_27(names_reg[0])
from scripts.get_vocab import *

def get_multi_loader(trains, valids, tests, config):
    names = config['prop_names']
    vocab = None if 'vocab' not in config else config['vocab']
    batch_size = config['batch_size']
    model_type = config['model_type']

    print('---> loader for', names)
    params_ = {'batch_size': batch_size, 'shuffle': True,
               'drop_last': False, 'num_workers': 0}
    param_t = {'batch_size': batch_size, 'shuffle': False,
               'drop_last': False, 'num_workers': 0}
    if model_type == 'RNN'and vocab == None:
        df = pd.concat([trains, valids, tests], ignore_index=True, axis=0)
        vocab = get_vocab(df)
    train_loader = get_loader(trains, names, params_, model_type, vocab)
    valid_loader = get_loader(valids, names, params_, model_type, vocab)
    test_loader  = get_loader(tests,  names, param_t, model_type, vocab)
    return train_loader, valid_loader, test_loader, vocab



VOCAB_TYPE: smiles, could change from ['char', 'smiles', 'selfies'] at get_vocab.py


### Params

In [ ]:
### CONSTANTS ###
names_reg = ['Caco2_Wang', 'Lipophilicity_AstraZeneca',
         'HydrationFreeEnergy_FreeSolv', 'Solubility_AqSolDB', 'LD50_Zhu'] # regression task
names_cls = ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
             'CYP1A2_Veith', 'CYP2C9_Veith'] + \
            ['BBB_Martins', 'Bioavailability_Ma',
             'Pgp_Broccatelli', 'HIA_Hou','PAMPA_NCATS'] + \
            ['hERG_Karim', 'AMES']

names_dict = {}
for name in names_reg + names_cls:
    if name in names_reg:   names_dict[name] = True  # regression task
    elif name in names_cls: names_dict[name] = False # classification task
names_all = list(names_dict.keys())

model_types = ['MLP', 'AttentiveFP', 'GIN', 'RNN']

In [117]:
in_dim = 256
hid_dims = [128, 64, 16]
dropout = 0.5
lr = 3e-4
wd = 1e-5
MAX_EPOCH = 1000
patience = 30           # stop if loss no decrease after epochs # patience
verbose_frequency = 100 # print evaluation every # verbose_frequency epoch
batch_size = 128

# special for AttentiveFP
graph_feat_size = 300
n_layers = 5
num_timesteps = 1   # times of updating the graph representations with GRU

# special for GIN: pretrain model types for selection:
pre_models_GIN = ['gin_supervised_contextpred', 'gin_supervised_infomax',
                     'gin_supervised_edgepred', 'gin_supervised_masking']
pre_model_num = 0    # choose from pre_models for GIN


# if VOCAB_TYPE == 'smiles':
import yaml
def load_vocab(VOCAB_TYPE):
    try:
        with open(f'vocab/{VOCAB_TYPE}.yml', 'r') as f: data = yaml.safe_load(f)
        vocab = data['vocab']; assert VOCAB_TYPE == data['vocab_type']
    except: vocab = None
    return vocab

# special for RNN:
VOCAB = load_vocab(VOCAB_TYPE)
Bid = True
GRU_num_layers = 3
GRU_dim = 256



scale_dict = None

def get_config(model_type, names,
               pre_model_num=pre_model_num, scale_dict=scale_dict):
    """
    Get config to initialize model
        param model_type: str, ['MLP', 'AttentiveFP', 'GIN', 'RNN']
        param names: list, task names
        param scale_dict: dict,
            if the task is regression, could scale label values
                            {name: [value_min, value_max], ...}
        param pre_model_num: int, [0, 1, 2, 3]
            if model_type is 'GIN', 4 types of pretrained models to choose from
    Returns config that could be used as PRED(**config)
    """
    pre_models_GIN = ['gin_supervised_contextpred', 'gin_supervised_infomax',
                         'gin_supervised_edgepred', 'gin_supervised_masking']

    # print(scale_dict)
    if isinstance(names, str): names = [names]
    IS_R = [names_dict[name] for name in names]
    config_MLP = {'model_type': 'MLP',
            'in_dim': 167,
            'hid_dims': hid_dims,
            'out_dim': len(names),
            'prop_names': names,
            'dropout': dropout,
            'IS_R': IS_R,
            'batch_size': batch_size,
            'lr': lr*2, # due to the simplicity of MLP, use larger lr
            'wd': wd,
            'patience': patience,
            'verbose_freq': verbose_frequency,
            'model_path': f'ckpt_MLP.pt',
            'scale_dict': scale_dict}

    config_ATF = {'model_type': 'AttentiveFP',
            'graph_feat_size': graph_feat_size,
            'num_timesteps': num_timesteps,
            'n_layers': n_layers,
            'out_dim': len(names),
            'prop_names': names,
            'dropout': dropout,
            'IS_R': IS_R,
            'batch_size': batch_size,
            'lr': lr,
            'wd': wd,
            'patience': patience,
            'verbose_freq': verbose_frequency,
            'model_path': 'ckpt_AT.pt',
            'scale_dict': scale_dict}

    config_GIN = {'model_type': 'GIN',
            'pretrain_model': pre_models_GIN[pre_model_num],
            'in_dim': in_dim,
            'hid_dims': hid_dims,
            'out_dim': len(names),
            'prop_names': names,
            'dropout': dropout,
            'batch_size': batch_size,
            'IS_R': IS_R,
            'lr': lr,
            'wd': wd,
            'patience': patience,
            'verbose_freq': verbose_frequency,
            'model_path': f'ckpt_GIN_{pre_models_GIN[pre_model_num]}.pt',
            'scale_dict': scale_dict}

    config_RNN = {'model_type': 'RNN',
              'vocab': VOCAB,
              'vocab_type': VOCAB_TYPE,
              'Bidirect': Bid,
              'num_layers': GRU_num_layers,
              'GRU_dim': GRU_dim,
              'out_dim': len(names),
              'prop_names': names,
              'dropout': dropout,
              'IS_R': IS_R,
              'device': 'cuda',
              'batch_size': batch_size,
              'lr': lr,
              'wd': wd,
              'patience': patience,
              'verbose_freq': verbose_frequency,
              'model_path': f'ckpt_RNN_{VOCAB_TYPE}.pt',
              'scale_dict': scale_dict}

    if model_type == 'MLP':           con_MO = config_MLP
    elif model_type == 'AttentiveFP': con_MO = config_ATF
    elif model_type == 'GIN':         con_MO = config_GIN
    elif model_type == 'RNN':         con_MO = config_RNN
    else: print('Error !{MLP, AttentiveFP, GIN, RNN}'); return None
    con_MO['config_path'] = con_MO['model_path'].split('.')[0] + '.yml'
    # different weight of task, initial weight the same
    con_MO['weight_loss'] = [float(1.0/len(names))] * len(names)
    con_MO['MAX_EPOCH'] = MAX_EPOCH
    return con_MO


import yaml
def yml_report(yml_path, recalculate=False, ver=False):
    """
    given yml path or yml data, return the test set performance
    param
        yml_path        : str,  the path of yml file
        recalculate     : bool, if true, will calculate from scratch
        ver             : bool, if true, will print detailed configs
    return perfm_dict   : dict, contain performance and test loss
    """
    if isinstance(yml_path, str): # the path string was input
        with open(yml_path, 'r') as f: data = yaml.safe_load(f)
    elif isinstance(yml_path, dict):   data = yml_path # data was input

    config = data['config']
    model_type = config['model_type']
    task_names = config['prop_names']
    perfm_dict =  data['performance']

    if len(perfm_dict) != 0 and recalculate == False:
        # during training, has evaluted test set, no need to calculate again
        # However for regression, the pred vs true value for test is not saved
        # if need the regression pred vs true value graph, need recalculate
        if ver: # print model config, and the training saved info
            print('#'*68); print('#'*30, 'CONFIG', '#'*30); print('#'*68)
            for i, j in config.items(): print(i, ':', j)
            print('#'*68)
            print('Model parameters: ', data['params_num'])
            times_list = data['times_list']
            print(f'Train time: {np.mean(times_list):.3f}'
                  f'+/-{np.std(times_list):.3f} ms')
            print(f"best epoch: {data['best_epoch']}, ",
                  f"min loss: {data['min_loss']:.4f}")
            plot_loss(data['train_dict'], data['valid_dict'], name='valid',
                      title_name= f'loss during training {model_type}')

    else: # recalculate from scratch using test data
        vocab = None if 'vocab' not in config else config['vocab']
        trains, valids, tests = collect_data_10_27(task_names)
        if config['scale_dict'] != None: # scale is done
            trains, valids, tests, dict_scale = scale(trains, valids, tests)
            assert config['scale_dict'] == dict_scale
        batch_size = config['batch_size']
        param_t = {'batch_size': batch_size, 'shuffle': False,
                'drop_last': False, 'num_workers': 0}
        test_loader = get_loader(tests, task_names, param_t, model_type, vocab)
        models = PRED(**config); models.load_status(data)
        outputs = models.eval(test_loader, ver=ver)
        perfm_dict = outputs[0]
    return perfm_dict
# def yml_report(yml_path, recalculate=False, ver=False):
#     """
#     given yml path, return the test set performance
#     param
#         yml_path        : str,  the path of yml file
#         recalculate     : bool, if true, will calculate from scratch
#         ver             : bool, if true, will print detailed configs
#     return perfm_dict   : dict, contain performance and test loss
#     """
#     with open(yml_path, 'r') as f: data = yaml.safe_load(f)
#     config = data['config']
#     model_type = config['model_type']
#     task_names = config['prop_names']
#     perfm_dict =  data['performance']

#     if len(perfm_dict) != 0 and recalculate == False:
#         # during training, has evaluted test set, no need to calculate again
#         # However for regression, the pred vs true value for test is not saved
#         # if need the regression pred vs true value graph, need recalculate
#         if ver: # print model config, and the training saved info
#             print('#'*68); print('#'*30, 'CONFIG', '#'*30); print('#'*68)
#             for i, j in config.items(): print(i, ':', j)
#             print('#'*68)
#             print('Model parameters: ', data['params_num'])
#             times_list = data['times_list']
#             print(f'Train time: {np.mean(times_list):.3f}'
#                   f'+/-{np.std(times_list):.3f} ms')
#             print(f"best epoch: {data['best_epoch']}, min loss: {data['min_loss']:.4f}")
#             plot_loss(data['train_dict'], data['valid_dict'], name='valid',
#                       title_name= f'loss during training {model_type}')
#         # return perfm_dict

#     else: # recalculate from scratch using test data
#         vocab = None if 'vocab' not in config else config['vocab']
#         trains, valids, tests = collect_data_10_27(task_names)
#         if config['scale_dict'] != None: # scale is done
#             trains, valids, tests, dict_scale = scale(trains, valids, tests)
#             assert config['scale_dict'] == dict_scale
#         batch_size = config['batch_size']
#         param_t = {'batch_size': batch_size, 'shuffle': False,
#                 'drop_last': False, 'num_workers': 0}
#         test_loader = get_loader(tests, task_names, param_t, model_type, vocab)
#         models = PRED(**config); models.load_status(data)
#         outputs = models.eval(test_loader, ver=ver)
#         perfm_dict = outputs[0]
#     return perfm_dict

import numpy as np
cls_metrics = ['acc', 'w_acc', 'prec', 'recall', 'sp', 'f1', 'auc', 'mcc', 'ap']
reg_metrics = ['mae', 'mse', 'rmse', 'r2']

# evaluate performance list, if directly from yml_file saved performance (dict)
# need to [yml_file_data['performance']] to convert dict into list
def eval_perf_list(perfs:list, name:list,
                   reg_metrics=reg_metrics,
                   cls_metrics=cls_metrics):
    """
    The same model type for multiple times, performance saved in list perfs
    Aim: evaluate performance of name, calculate mean and std for multiple run
    """

    repeat_time = len(perfs) # the same model was run for # repeat_time times
    if repeat_time > 1: # multiple run, find the lowest loss
        loss_list = [p['loss'] for p in perfs]
        best_model_idx = np.argmin(loss_list) # has the lowest loss
        # print(f'idx {best_model_idx} has the lowest loss')
        best_perf = perfs[best_model_idx]
    else: best_perf = None
    for n in name:
        IS_R = names_dict[n]
        # print(n, '\tRegression?', IS_R);
        if IS_R: ms = reg_metrics # is regression,     use reg_metrics (#4)
        else:    ms = cls_metrics # is classification, use cls_metrics (#9)
        results = {}
        for idx, i in enumerate(perfs): # access idx_th evaluation in perfs
            r = i[n]; results[idx] = r  # collect the evaluation for name n

        means, stds = [], []
        print('*'*20, n, '*'*20, end='\n\t')
        for k in ms: print('\t', k, end = ' \t ')
        print()
        for idx_v in range(len(r)):
            cur_values = []
            for idx in range(repeat_time):
                cur_v = results[idx][idx_v]; cur_values.append(cur_v)
            mean_here, std_here = np.mean(cur_values), np.std(cur_values)
            means.append(mean_here); stds.append(std_here)
            # print(f'{ms[idx_v]}\t: {mean_here:.3f} +/- {std_here:.3f}')

        for idx_final, (i, j) in enumerate(zip(means, stds)):
            if idx_final == 0: print(end='\t| ')
            print(f'{i:.3f} +/- {j:.3f}', end=' | ')
        if best_perf != None:
            print(f'\n idx {best_model_idx}: ', end='| ')
            for i in range(len(ms)):
                print(f'{best_perf[n][i]:.3f} +/- {0:.3f}', end=' | ')
        print('\n\n')


        # break


In [118]:
repeat_time = 3
name = ['Lipophilicity_AstraZeneca', 'Solubility_AqSolDB'] # have run MT on those 2 tasks

for folder_ in ['no_scale', 'scale']:
    print(folder_)
    for model_type in model_types:
        try:
            print(model_type)
            yml_files = [f'{folder_}/{model_type}_MT_{i}.yml' for i in range(repeat_time)]
            perform_list = [yml_report(yml_file, ver=False) for yml_file in yml_files]
            eval_perf_list(perform_list, name)
        except: pass # no yml files saved yet, skip
    print()


no_scale
MLP
******************** Lipophilicity_AstraZeneca ********************
		 mae 	 	 mse 	 	 rmse 	 	 r2 	 
	| 0.777 +/- 0.036 | 0.951 +/- 0.065 | 0.975 +/- 0.033 | 0.356 +/- 0.044 | 
 idx 1: | 0.746 +/- 0.000 | 0.885 +/- 0.000 | 0.941 +/- 0.000 | 0.401 +/- 0.000 | 


******************** Solubility_AqSolDB ********************
		 mae 	 	 mse 	 	 rmse 	 	 r2 	 
	| 0.930 +/- 0.022 | 1.603 +/- 0.073 | 1.266 +/- 0.029 | 0.705 +/- 0.013 | 
 idx 1: | 0.910 +/- 0.000 | 1.524 +/- 0.000 | 1.235 +/- 0.000 | 0.719 +/- 0.000 | 


AttentiveFP
******************** Lipophilicity_AstraZeneca ********************
		 mae 	 	 mse 	 	 rmse 	 	 r2 	 
	| 0.480 +/- 0.008 | 0.402 +/- 0.010 | 0.634 +/- 0.008 | 0.728 +/- 0.007 | 
 idx 0: | 0.477 +/- 0.000 | 0.406 +/- 0.000 | 0.637 +/- 0.000 | 0.725 +/- 0.000 | 


******************** Solubility_AqSolDB ********************
		 mae 	 	 mse 	 	 rmse 	 	 r2 	 
	| 0.660 +/- 0.007 | 0.963 +/- 0.029 | 0.981 +/- 0.015 | 0.823 +/- 0.005 | 
 idx 0: | 0.650 +/- 0.

In [ ]:
for i, j in names_dict.items(): print(i, ': \t Regression? :', j)

print('\nVOCAB TYPE for RNN:', VOCAB_TYPE)

Caco2_Wang : 	 Regression? : True
Lipophilicity_AstraZeneca : 	 Regression? : True
HydrationFreeEnergy_FreeSolv : 	 Regression? : True
Solubility_AqSolDB : 	 Regression? : True
LD50_Zhu : 	 Regression? : True
CYP2C19_Veith : 	 Regression? : False
CYP2D6_Veith : 	 Regression? : False
CYP3A4_Veith : 	 Regression? : False
CYP1A2_Veith : 	 Regression? : False
CYP2C9_Veith : 	 Regression? : False
BBB_Martins : 	 Regression? : False
Bioavailability_Ma : 	 Regression? : False
Pgp_Broccatelli : 	 Regression? : False
HIA_Hou : 	 Regression? : False
PAMPA_NCATS : 	 Regression? : False
hERG_Karim : 	 Regression? : False
AMES : 	 Regression? : False

VOCAB TYPE for RNN: smiles


In [119]:
##########################################################
# test scale importance on Regression dataset
# Specify task name,    whether scale dataset,
# repeat how many times on the same model type

# name = ['Lipophilicity_AstraZeneca', 'Solubility_AqSolDB']
name = ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
        'CYP1A2_Veith', 'CYP2C9_Veith']
scale_dataset = False
repeat_time = 3 # run the same model # repeat_time times
##########################################################

if len(name) > 1: run_type = 'MT' # multi task
else: run_type = 'ST' # single task
model_types = ['MLP', 'AttentiveFP', 'GIN', 'RNN']

trn, val, tst = collect_data_10_27(name) # collect data for task: name

if scale_dataset: # choose to scale dataset
    trn, val, tst, dict_scale = scale(trn, val, tst); folder_name = 'scale'
else: dict_scale = None; folder_name = 'no_scale'

folder_name = 'M5' # 5 metabolism tasks

make_path(folder_name, False)

# loop all model types
for model_type in model_types[3:]:
    config = get_config(model_type, name); config['scale_dict'] = dict_scale
    trn_l, val_l, tst_l, vocab = get_multi_loader(trn, val, tst, config)
    if vocab != None and config['vocab'] == None:
        """ If need vocab (RNN) yet no vocab provided: calculate using dataset """
        config['vocab'] = vocab # update config vocab info
        print('RNN, no vocab provided, update vocab using dataset | ', end="")
        print('vocab length updated:', len(vocab))

    perfs = [] # a list to store the performance outputted from the model
    for i in range(repeat_time): # run the same model for repeat_time times
        save_dir = f'{folder_name}/{model_type}_{run_type}_{i}'
        config['model_path'] = save_dir + '.pt'
        config['config_path'] = save_dir + '.yml'

        print(f'\nRun # {i} for {model_type} {run_type}', end='\t')
        try: # try open yml file, if file exists, and no need train
            with open(config['config_path'], 'r') as f: data = yaml.safe_load(f)
            if data != None: p = yml_report(data); print('--> pre data loaded')
        except:  # model was not trained yet, train the model
            models = PRED(**config); p = models.train(trn_l, val_l, tst_l)
        perfs.append(p)

    print('\n', model_type)
    eval_perf_list(perfs, name) # print the mean+/-std for model_type


Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...


collect data for:  ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith', 'CYP1A2_Veith', 'CYP2C9_Veith']


Loading...
Done!


---> loader for ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith', 'CYP1A2_Veith', 'CYP2C9_Veith']
--> preparing data loader for model type  RNN
--> preparing data loader for model type  RNN
--> preparing data loader for model type  RNN

Run # 0 for RNN MT	--> pre data loaded

Run # 1 for RNN MT	--> pre data loaded

Run # 2 for RNN MT	--> pre data loaded

 RNN
******************** CYP2C19_Veith ********************
		 acc 	 	 w_acc 	 	 prec 	 	 recall 	 	 sp 	 	 f1 	 	 auc 	 	 mcc 	 	 ap 	 
	| 0.816 +/- 0.005 | 0.817 +/- 0.004 | 0.786 +/- 0.010 | 0.824 +/- 0.005 | 0.809 +/- 0.012 | 0.804 +/- 0.003 | 0.884 +/- 0.003 | 0.631 +/- 0.008 | 0.850 +/- 0.006 | 
 idx 1: | 0.819 +/- 0.000 | 0.819 +/- 0.000 | 0.794 +/- 0.000 | 0.818 +/- 0.000 | 0.820 +/- 0.000 | 0.806 +/- 0.000 | 0.886 +/- 0.000 | 0.637 +/- 0.000 | 0.856 +/- 0.000 | 


******************** CYP2D6_Veith ********************
		 acc 	 	 w_acc 	 	 prec 	 	 recall 	 	 sp 	 	 f1 	 	 auc 	 	 mcc 	 	 ap 	 
	| 0.858 +/- 0.004 | 0.692 +/- 0

In [106]:
repeat_time = 3
name = ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
        'CYP1A2_Veith', 'CYP2C9_Veith']

for folder_ in ['M5']: # 5 metabolism tasks trained together
    print(folder_)
    for model_type in model_types:
        try:
            print(model_type)
            yml_files = [f'{folder_}/{model_type}_MT_{i}.yml' for i in range(repeat_time)]
            perform_list = [yml_report(yml_file, ver=False) for yml_file in yml_files]
            loss_list = [p['loss'] for p in perform_list]
            best_model_idx = np.argmin(loss_list) # has the lowest loss
            print(f'model_type {model_type}, file with lowest loss:',
                 f'{folder_}/{model_type}_MT_{best_model_idx}.yml\n')
            eval_perf_list(perform_list, name)
            print('\n\n\n\n\n')
            # eval_perf_list([perform_list[best_model_idx]], name)
        except: pass # no yml files saved yet, skip
    print()


M5
MLP
model_type MLP, file with lowest loss: M5/MLP_MT_1.yml

******************** CYP2C19_Veith ********************
		 acc 		 w_acc 		 prec 		 recall 		 sp 		 f1 		 auc 		 mcc 		 ap 	
	| 0.774 +/- 0.007 | 0.774 +/- 0.008 | 0.747 +/- 0.008 | 0.768 +/- 0.014 | 0.779 +/- 0.009 | 0.757 +/- 0.009 | 0.845 +/- 0.005 | 0.546 +/- 0.015 | 0.800 +/- 0.008 | 
 idx 1: | 0.776 +/- 0.000 | 0.776 +/- 0.000 | 0.743 +/- 0.000 | 0.782 +/- 0.000 | 0.770 +/- 0.000 | 0.762 +/- 0.000 | 0.847 +/- 0.000 | 0.551 +/- 0.000 | 0.805 +/- 0.000 | 


******************** CYP2D6_Veith ********************
		 acc 		 w_acc 		 prec 		 recall 		 sp 		 f1 		 auc 		 mcc 		 ap 	
	| 0.837 +/- 0.004 | 0.568 +/- 0.018 | 0.905 +/- 0.063 | 0.140 +/- 0.039 | 0.996 +/- 0.004 | 0.239 +/- 0.055 | 0.807 +/- 0.013 | 0.313 +/- 0.028 | 0.566 +/- 0.026 | 
 idx 1: | 0.843 +/- 0.000 | 0.593 +/- 0.000 | 0.826 +/- 0.000 | 0.195 +/- 0.000 | 0.991 +/- 0.000 | 0.315 +/- 0.000 | 0.815 +/- 0.000 | 0.352 +/- 0.000 | 0.581 +/- 0.000 | 


********